# Baseline A: Tìm kiếm độc lập từng Layer (Layer-wise) bằng Faiss L2 trên vector đã “làm trắng” (1 test -  top k - 5 train)

## Mục tiêu của script
Script này thực hiện một baseline retrieval theo kiểu **layer-wise độc lập**, tức là:
- Mỗi layer (1 → 12) được xem như một không gian embedding riêng.
- Với **mỗi layer**:
  1. Xây dựng **Faiss IndexFlatL2** từ **Train DB đã whiten** (Bước 2b).
  2. Với các query (Test DB gốc):
     - **Whiten query** bằng mean + whitening matrix của layer (từ Bước 2a).
     - Search Top-K theo **L2 (Euclidean)** trên index của layer đó.
  3. Lưu kết quả của layer thành **một file JSON riêng**.

Kết quả cuối cùng:
- Có **12 file kết quả**, mỗi file tương ứng `layer_1 ... layer_12`.


## Thư viện sử dụng
- `torch`: xử lý tensor, chạy GPU nếu có, whitening query bằng matmul.
- `os`: thao tác đường dẫn, tạo thư mục output.
- `json`: đọc metadata và ghi kết quả JSON.
- `numpy`: chuyển tensor sang numpy float32, sqrt khoảng cách.
- `tqdm.auto`: progress bar (batch query).
- `sys`, `subprocess`: phục vụ cài đặt Faiss tự động.


## 1) Hàm `check_and_install_faiss()`

### Mục đích
- Kiểm tra xem môi trường đã có `faiss` chưa.
- Nếu chưa có:
  - tự động cài đặt `faiss-gpu` nếu có CUDA
  - hoặc `faiss-cpu` nếu không có GPU
- Sau khi cài xong sẽ cảnh báo **phải restart runtime/session**.

### Luồng xử lý
1. `try: import faiss`
   - Nếu import được → trả `True`.

2. Nếu `ImportError`:
   - Chọn package:
     - GPU: `faiss-gpu`
     - CPU: `faiss-cpu`
   - Cài qua pip bằng:
     - `subprocess.check_call([sys.executable, "-m", "pip", "install", package])`
   - Nếu cài thành công:
     - in cảnh báo restart runtime
     - trả `False` (để dừng và chạy lại sau restart)
   - Nếu cài lỗi:
     - in lỗi và trả `False`.

## 2) Hàm chính `main()`

### Mục tiêu tổng quát
- Chạy tìm kiếm layer-wise cho 12 layer.
- Mỗi layer:
  - build index từ train whitened
  - whiten query của layer đó
  - search Top-K L2
  - lưu JSON riêng cho layer

## 2.1 Cấu hình chính trong `main()`
- `K = 10`  
  Số hàng xóm gần nhất (Top-K) cho mỗi query trên mỗi layer.

- `D = 2304`  
  Dimension của embedding.

- `NUM_LAYERS = 12`  
  Số layer xử lý.

- `device = cuda/cpu`  
  Nếu có GPU → sử dụng CUDA để whiten query nhanh hơn.

- `use_gpu = torch.cuda.is_available()`  
  Nếu `True` thì Faiss index sẽ được chuyển sang GPU.


## 2.2 Thiết lập đường dẫn (Baseline A)
### Thư mục output
- `search_results/layer_wise_results_baseline_A`
- Mỗi layer sẽ tạo một file:
  - `search_results_layer_1.json`
  - ...
  - `search_results_layer_12.json`

### Input files
- **Train vectors (đã whiten)**:
  - `combined_train_db_vectors_v2_WHITENED.pt`
- **Train metadata**:
  - `combined_train_db_metadata_v2_inner_content.json`

- **Test vectors (gốc)**:
  - `combined_TEST_db_vectors_v2_inner_content.pt`
- **Test metadata**:
  - `combined_TEST_db_metadata_v2_inner_content.json`

- **Whitening matrices (mean + W)**:
  - `mahalanobis_data_v2_inner_content.pt`

## 3) Tải dữ liệu
Script load:
- `train_db`: tensor `[N, 12, 2304]` (whitened)
- `train_meta`: list metadata length `N`
- `test_db`: tensor `[M, 12, 2304]` (original)
- `test_meta`: list metadata length `M`
- `mahal_data`: dict mean + W theo layer (load lên `device`)

Sau đó:
- `N = len(train_meta)`
- `num_queries = len(test_meta)`

Nếu load lỗi → in lỗi và `return`.


## 4) Vòng lặp xử lý từng layer (layer-wise)
Thay vì build 12 index cùng lúc (tốn RAM), script làm:
- **build index → search → lưu file → giải phóng**
cho từng layer.

Với mỗi `layer_idx` (0..11):
- `layer_name = f"layer_{layer_idx+1}"`


## 4.1 A) Xây dựng Faiss Index cho layer hiện tại
1. Trích vector train của layer:
   - `vecs = train_db[:, layer_idx, :].numpy().astype('float32')`
   - Shape: `[N, 2304]`

2. Tạo index L2:
   - `index = faiss.IndexFlatL2(D)`

3. Nếu `use_gpu`:
   - tạo resource:
     - `res = faiss.StandardGpuResources()`
   - chuyển index sang GPU:
     - `index = faiss.index_cpu_to_gpu(res, 0, index)`

4. Add vectors:
   - `index.add(vecs)`

## 4.2 B) Tìm kiếm cho layer hiện tại (batch query)
Chuẩn bị:
- `layer_results = []` để lưu kết quả của layer.

Lấy mean/W của layer:
- `mean = mahal_data['means'][layer_name]`  shape `(2304,)`
- `W = mahal_data['whitening_matrices'][layer_name]` shape `(2304, 2304)`

Thiết lập batch:
- `BATCH_SIZE = 128`
- Loop `start` từ `0` đến `num_queries` bước `BATCH_SIZE`.

### Bước tìm kiếm theo batch
Với batch `[start:end]`:

#### (1) Lấy query vectors của layer
- `q_batch = test_db[start:end, layer_idx, :].to(device)`
- Shape: `[B, 2304]`

#### (2) Whitening query
- Center:
  - `q_batch_centered = q_batch - mean`
- Whiten:
  - `q_batch_whitened = torch.matmul(q_batch_centered, W.T)`
- Đưa về numpy float32:
  - `q_batch_np = q_batch_whitened.cpu().numpy().astype('float32')`

> Lưu ý: dùng `W.T` để phù hợp convention row-vector: `(q-mean) @ W^T`

#### (3) Search Top-K bằng Faiss
- `D_sq, I = index.search(q_batch_np, K)`

Trong đó:
- `D_sq`: khoảng cách L2 **bình phương** shape `[B, K]`
- `I`: chỉ số neighbor tương ứng shape `[B, K]`

#### (4) Lưu kết quả vào JSON-friendly format
Với từng query trong batch:
- `query_real_idx = start + i`
- Tạo list `neighbors`:
  - `match_db_index`: chỉ số trong train
  - `l2_distance`: `sqrt(D_sq)` để ra L2 thật
  - `match_metadata`: metadata của train tương ứng

Lưu vào `layer_results`:
- `query_info`: `test_meta[query_real_idx]`
- `neighbors`: list Top-K

## 4.3 C) Lưu file kết quả cho layer
Sau khi xong search cho layer:
- `output_file = search_results_layer_{k}.json`
- `json.dump(layer_results, indent=4)`

## 4.4 Giải phóng bộ nhớ
- `del index`
- Nếu dùng GPU:
  - `torch.cuda.empty_cache()`

Mục tiêu:
- tránh tăng VRAM/RAM theo thời gian khi xử lý 12 layer.

## 5) Kết thúc chương trình
Sau khi chạy đủ 12 layer:
- In thông báo hoàn tất
- Tất cả kết quả nằm trong:
  - `search_results/layer_wise_results_baseline_A/`

## 6) Entry point (`__main__`)
- Trước tiên gọi:
  - `check_and_install_faiss()`
- Nếu trả `True` (đã có Faiss) → chạy `main()`
- Nếu trả `False` (vừa cài xong hoặc lỗi cài) → dừng để người dùng restart hoặc xử lý lỗi.

In [ ]:
import torch
import os
import json
import numpy as np
from tqdm.auto import tqdm
import sys
import subprocess

# --- HÀM CÀI ĐẶT FAISS ---
def check_and_install_faiss():
    try:
        import faiss
        print("Đã tìm thấy thư viện Faiss.")
        return True
    except ImportError:
        print("Chưa tìm thấy Faiss. Đang tự động cài đặt...")
        package = "faiss-gpu" if torch.cuda.is_available() else "faiss-cpu"
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"Cài đặt {package} hoàn tất.")
            print("\nCẢNH BÁO: Vui lòng KHỞI ĐỘNG LẠI RUNTIME (Restart Session) rồi chạy lại file này.")
            return False
        except Exception as e:
            print(f"Lỗi cài đặt: {e}")
            return False



# --- HÀM CHÍNH ---
def main():
    import faiss

    print("--- Bắt đầu Tìm kiếm Độc lập từng Layer (Baseline A - Layer-wise) ---")

    # CẤU HÌNH
    K = 10
    D = 2304
    NUM_LAYERS = 12
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    use_gpu = torch.cuda.is_available()
    print(f"Sử dụng thiết bị: {device} (Faiss GPU: {use_gpu})")

    # THIẾT LẬP ĐƯỜNG DẪN (BASELINE A)
    drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'
    db_cache_dir = os.path.join(drive_base_path, 'cached_databases')

    # Thư mục lưu kết quả layer-wise
    output_results_dir = os.path.join(drive_base_path, 'search_results/layer_wise_results_baseline_A')
    os.makedirs(output_results_dir, exist_ok=True)

    # *** INPUT: Dùng file DB đã 'làm trắng' của Baseline A ***
    train_vectors_file = os.path.join(db_cache_dir, 'combined_train_db_vectors_v2_WHITENED.pt')
    train_metadata_file = os.path.join(db_cache_dir, 'combined_train_db_metadata_v2_inner_content.json')

    test_vectors_file = os.path.join(db_cache_dir, 'combined_TEST_db_vectors_v2_inner_content.pt')
    test_metadata_file = os.path.join(db_cache_dir, 'combined_TEST_db_metadata_v2_inner_content.json')

    mahalanobis_data_file = os.path.join(db_cache_dir, 'mahalanobis_data_v2_inner_content.pt')

    print(f"Input Train DB: {train_vectors_file}")
    print(f"Output Dir: {output_results_dir}")

    # TẢI DỮ LIỆU
    print("\nĐang tải dữ liệu...")
    try:
        train_db = torch.load(train_vectors_file, map_location='cpu')
        with open(train_metadata_file, 'r') as f: train_meta = json.load(f)

        test_db = torch.load(test_vectors_file, map_location='cpu')
        with open(test_metadata_file, 'r') as f: test_meta = json.load(f)

        mahal_data = torch.load(mahalanobis_data_file, map_location=device)

        N = len(train_meta)
        print(f" -> Train DB: {N} mẫu")
        print(f" -> Test DB: {len(test_meta)} mẫu")
    except Exception as e:
        print(f"LỖI TẢI DỮ LIỆU: {e}")
        return

    # XỬ LÝ TỪNG LAYER
    # Xây dựng index từng cái

    for layer_idx in range(NUM_LAYERS):
        layer_name = f"layer_{layer_idx+1}"
        print(f"\n>>> Đang xử lý {layer_name} ...")

        # Xây dựng Index cho Layer này
        vecs = train_db[:, layer_idx, :].numpy().astype('float32')
        index = faiss.IndexFlatL2(D)
        if use_gpu:
            res = faiss.StandardGpuResources()
            index = faiss.index_cpu_to_gpu(res, 0, index)
        index.add(vecs)

        # Tìm kiếm cho Layer này
        layer_results = []

        # Lấy ma trận làm trắng của layer này
        mean = mahal_data['means'][layer_name]
        W = mahal_data['whitening_matrices'][layer_name]

        # Batch processing cho query để nhanh hơn
        BATCH_SIZE = 128
        num_queries = len(test_meta)

        for start in tqdm(range(0, num_queries, BATCH_SIZE), desc=f"Search {layer_name}"):
            end = min(start + BATCH_SIZE, num_queries)

            # Lấy batch query vectors của layer này
            # test_db shape: [M, 12, 2304] -> lấy [batch, layer_idx, :]
            q_batch = test_db[start:end, layer_idx, :].to(device)

            # Làm trắng (Whiten)
            # (q - mean) @ W.T
            q_batch_centered = q_batch - mean
            q_batch_whitened = torch.matmul(q_batch_centered, W.T)
            q_batch_np = q_batch_whitened.cpu().numpy().astype('float32')

            # Tìm kiếm Top-K
            # D_sq: Khoảng cách bình phương, I: Indices
            D_sq, I = index.search(q_batch_np, K)

            # Lưu kết quả
            for i in range(len(q_batch_np)):
                query_real_idx = start + i

                neighbors = []
                for k in range(K):
                    match_idx = I[i][k]
                    dist = float(np.sqrt(D_sq[i][k]))

                    neighbors.append({
                        'match_db_index': int(match_idx),
                        'l2_distance': dist,
                        'match_metadata': train_meta[match_idx]
                    })

                layer_results.append({
                    'query_info': test_meta[query_real_idx],
                    'neighbors': neighbors
                })

        # Lưu file kết quả cho Layer này
        output_file = os.path.join(output_results_dir, f'search_results_{layer_name}.json')
        with open(output_file, 'w') as f:
            json.dump(layer_results, f, indent=4)
        print(f" -> Đã lưu kết quả {layer_name}: {output_file}")

        # Giải phóng bộ nhớ GPU
        del index
        if use_gpu: torch.cuda.empty_cache()

    print(f"\n Toàn bộ 12 file kết quả đã được lưu tại:\n{output_results_dir}")

if __name__ == "__main__":
    if check_and_install_faiss():
        main()

✅ Đã tìm thấy thư viện Faiss.
--- Bắt đầu Tìm kiếm Độc lập từng Layer (Baseline A - Layer-wise) ---
Sử dụng thiết bị: cpu (Faiss GPU: False)
Input Train DB: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/cached_databases/combined_train_db_vectors_v2_WHITENED.pt
Output Dir: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/search_results/layer_wise_results_baseline_A

Đang tải dữ liệu...
 -> Train DB: 15521 mẫu
 -> Test DB: 2410 mẫu

>>> Đang xử lý layer_1 ...


Search layer_1:   0%|          | 0/19 [00:00<?, ?it/s]

 -> Đã lưu kết quả layer_1: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/search_results/layer_wise_results_baseline_A/search_results_layer_1.json

>>> Đang xử lý layer_2 ...


Search layer_2:   0%|          | 0/19 [00:00<?, ?it/s]

 -> Đã lưu kết quả layer_2: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/search_results/layer_wise_results_baseline_A/search_results_layer_2.json

>>> Đang xử lý layer_3 ...


...

 -> Đã lưu kết quả layer_10: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/search_results/layer_wise_results_baseline_A/search_results_layer_10.json

>>> Đang xử lý layer_11 ...


Search layer_11:   0%|          | 0/19 [00:00<?, ?it/s]

 -> Đã lưu kết quả layer_11: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/search_results/layer_wise_results_baseline_A/search_results_layer_11.json

>>> Đang xử lý layer_12 ...


Search layer_12:   0%|          | 0/19 [00:00<?, ?it/s]

 -> Đã lưu kết quả layer_12: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/search_results/layer_wise_results_baseline_A/search_results_layer_12.json

✅ HOÀN TẤT! Toàn bộ 12 file kết quả đã được lưu tại:
/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/search_results/layer_wise_results_baseline_A
